In [45]:
pip install anchor-exp

In [ ]:
pip install torch transformers spacy && python -m spacy download en_core_web_sm


In [47]:
%load_ext autoreload
%autoreload 2
import os
import os.path
import numpy as np
import sklearn
import sklearn.model_selection
import sklearn.linear_model
import sklearn.ensemble
import spacy
import sys
from sklearn.feature_extraction.text import CountVectorizer
from anchor import anchor_text
import time

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [48]:
import pandas as pd


In [49]:
nlp = spacy.load('en_core_web_sm')

In [51]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert").cuda()

In [52]:
import torch
import scipy as sp


In [53]:
explainer = anchor_text.AnchorText(nlp, ['negative', 'neutral', 'positive'], use_unk_distribution=True)

In [54]:
def f(x):
    tv = torch.tensor([tokenizer.encode(v, padding='max_length', max_length=500, truncation=True) for v in x]).cuda()
    outputs = model(tv)[0].detach().cpu().numpy()
    scores = (np.exp(outputs).T / np.exp(outputs).sum(1)).T
    val = sp.special.logit(scores[:,1]) # use one vs rest logit units
    return val

In [55]:
def f2(x):
    tv = torch.tensor([tokenizer.encode(v, padding='max_length', max_length=500, truncation=True) for v in x]).cuda()
    outputs = model(tv)[0].detach().cpu().numpy()
    scores = (np.exp(outputs).T / np.exp(outputs).sum(1)).T
    val = sp.special.logit(scores[:,1]) # use one vs rest logit units
    return int(val)

In [56]:
import pandas as pd

In [57]:
df=pd.read_csv("dataSpecial.csv", encoding="latin", header=[0])
desc=df['Description']

In [81]:
x=desc[3]
print(x)

Calls to the switchboard and directory services have decreased significantly since our employees now have up-to-date contact information from all their colleagues and customers on their phone and can place the call directly .


In [69]:
pip install gensim

In [82]:
from gensim.parsing.preprocessing import remove_stopwords

y = remove_stopwords(x)

print(y)


Calls switchboard directory services decreased significantly employees up-to-date contact information colleagues customers phone place directly .


In [83]:
pred = explainer.class_names[f2([y])]
print(pred)
exp = explainer.explain_instance(y, f, threshold=0.95)

negative


In [84]:
pred = explainer.class_names[f2([y])]
print(pred)
exp = explainer.explain_instance(y, f, threshold=0.95)
print('Anchor: %s' % (' *** '.join(exp.names())))
print('Examples where anchor applies and model predicts %s:' % pred)
print()
print('\n'.join([x[0] for x in exp.examples(only_same_prediction=True)]))
print()
print('\n'.join([x[0] for x in exp.examples(partial_index=0, only_different_prediction=True)]))

negative
Anchor: . *** to *** services *** employees *** directly *** date *** contact *** up *** Calls *** phone *** customers *** - *** colleagues *** switchboard *** information *** decreased *** - *** directory *** place *** significantly
Examples where anchor applies and model predicts negative:



Calls switchboard UNK UNK decreased significantly employees up UNK UNK - date UNK UNK UNK customers UNK place 
UNK UNK UNK UNK decreased significantly employees UNK - UNK UNK date contact UNK UNK customers UNK place direc
Calls UNK UNK UNK UNK UNK UNK UNK UNK to UNK UNK contact UNK colleagues UNK phone place directly .
Calls switchboard directory UNK UNK significantly UNK up UNK to - UNK contact information colleagues customers
UNK switchboard directory UNK decreased UNK UNK up UNK to - date contact information colleagues UNK UNK UNK UN
Calls UNK directory UNK UNK UNK employees up UNK to UNK UNK contact information colleagues UNK phone place dir
Calls switchboard UNK services decreased 

In [85]:
sent1=y.replace("decreased", "stayed the same")
print(sent1)

Calls switchboard directory services stayed the same significantly employees up-to-date contact information colleagues customers phone place directly .


In [86]:
pred = explainer.class_names[f2([sent1])]
print(pred)
exp = explainer.explain_instance(sent1, f, threshold=0.95)
print('Anchor: %s' % (' *** '.join(exp.names())))
print('Examples where anchor applies and model predicts %s:' % pred)
print()
print('\n'.join([x[0] for x in exp.examples(only_same_prediction=True)]))
print()
print('\n'.join([x[0] for x in exp.examples(partial_index=0, only_different_prediction=True)]))

positive
Anchor: . *** stayed *** switchboard *** - *** significantly *** date *** same *** to *** services *** Calls *** directly *** employees *** directory *** the *** phone *** information *** up *** customers *** contact *** - *** colleagues *** place
Examples where anchor applies and model predicts positive:



Calls switchboard UNK UNK stayed UNK same significantly UNK UNK - to UNK UNK contact UNK colleagues UNK UNK UNK UN
Calls switchboard directory UNK UNK UNK UNK UNK UNK up UNK UNK - date UNK information colleagues UNK phone place U
Calls UNK directory services UNK the UNK significantly employees UNK - to - UNK UNK UNK colleagues customers phone
Calls UNK UNK services UNK the UNK significantly UNK up UNK to UNK date contact information UNK UNK UNK place UNK 
Calls switchboard UNK services stayed the UNK significantly UNK up - UNK UNK date UNK UNK colleagues customers pho
Calls switchboard UNK UNK stayed the same significantly employees UNK - UNK - date contact UNK UNK UNK UNK

In [87]:
sent2=y.replace("decreased", "increased")
print(sent2)

Calls switchboard directory services increased significantly employees up-to-date contact information colleagues customers phone place directly .


In [88]:
pred = explainer.class_names[f2([sent2])]
print(pred)
exp = explainer.explain_instance(sent2, f, threshold=0.95)
print('Anchor: %s' % (' *** '.join(exp.names())))
print('Examples where anchor applies and model predicts %s:' % pred)
print()
print('\n'.join([x[0] for x in exp.examples(only_same_prediction=True)]))
print()
print('\n'.join([x[0] for x in exp.examples(partial_index=0, only_different_prediction=True)]))

positive
Anchor: . *** to *** services *** employees *** directly *** date *** contact *** up *** Calls *** phone *** customers *** - *** colleagues *** switchboard *** information *** increased *** - *** directory *** place *** significantly
Examples where anchor applies and model predicts positive:



UNK UNK directory UNK increased UNK UNK UNK - UNK UNK date contact information UNK customers UNK place dir
Calls switchboard directory UNK increased significantly employees UNK UNK to UNK UNK contact information c
UNK switchboard UNK UNK increased significantly UNK UNK - to - UNK UNK information colleagues UNK UNK UNK 
Calls switchboard UNK UNK increased significantly UNK up UNK UNK - date UNK UNK UNK UNK phone place UNK .
UNK switchboard directory services UNK UNK UNK UNK - to - date UNK information UNK UNK phone UNK UNK .
Calls UNK directory services increased UNK UNK up UNK to UNK date UNK information UNK customers phone plac
UNK switchboard directory UNK UNK significantly UNK up - t